In [1]:
#clear all
%reset -f

#import packages
import numpy as np
import time
import sys
import os
import pandas as pd
import mne
import matplotlib
from datetime import datetime
from mne_icalabel import label_components

#pop up plots as separate window & interactive
%matplotlib qt
matplotlib.pyplot.close('all')

root = 'F:/Documents/Science/MirRevAdaptEEG'

# Set the events
event_dict = {
        #trial triggers        
        "trial_onset": 16138,
        "target_onset": 16139,
        "go_target_onset": 16140,
        "reach_onset": 16141,
        "reach_end_feedback_onset": 16142, #reach end is when step moves to 5, which is also onset for feedback
        "feedback_home": 16143,
        "fixation_onset": 16144,
        #task triggers
        "task_rest": 16148,
        "task_aligned": 16149,
        "task_random0": 16150,
        "task_rotation": 16151,
        "task_washout0": 16152,
        "task_random1": 16153,
        "task_mirror": 16154,
        "task_washout1": 16155,
        "task_end": 16156,
        #target locations based on participant condition (only get 12 of below)
        "target_q1_hor_near": 16158,
        "target_q1_hor_mid": 16159,
        "target_q1_hor_far": 16160,
        "target_q3_hor_near": 16161,
        "target_q3_hor_mid": 16162,
        "target_q3_hor_far": 16163,
        "target_q1_ver_near": 16164,
        "target_q1_ver_mid": 16165,
        "target_q1_ver_far": 16166,
        "target_q3_ver_near": 16167,
        "target_q3_ver_mid": 16168,
        "target_q3_ver_far": 16169,

        "target_q2_hor_near": 16178,
        "target_q2_hor_mid": 16179,
        "target_q2_hor_far": 16180,
        "target_q4_hor_near": 16181,
        "target_q4_hor_mid": 16182,
        "target_q4_hor_far": 16183,
        "target_q2_ver_near": 16184,
        "target_q2_ver_mid": 16185,
        "target_q2_ver_far": 16186,
        "target_q4_ver_near": 16187,
        "target_q4_ver_mid": 16188,
        "target_q4_ver_far": 16189,

        #extra target info triggers come up in recording
        "targetinfo_1": 16175,
        "targetinfo_2": 16190,
        "targetinfo_3": 16191
}

In [2]:
#setting up path/ directory

#access specific file for specific participant
def load_bdf(pp_num, root_dir):

    root_directory = root_dir
    data_directory = os.path.join(root_directory, 'data/eeg/')
    pp_directory = os.path.join(data_directory, 'p%03d/' % pp_num)
    filename = os.path.join(pp_directory, 'mra_p%03d_run0.bdf' % pp_num)

    data = mne.io.read_raw_bdf(filename, preload = True, exclude = ['EXG3', 'EXG4', 'EXG5', 'EXG6', 'EXG7', 'EXG8'])
    return data, pp_directory

In [3]:
def set_reference(pp_num, root_dir):
    
    data, pp_directory = load_bdf(pp_num, root_dir)
    
    #use mastoids as reference
    data_ref = mne.io.Raw.set_eeg_reference(data, ref_channels = ['EXG1', 'EXG2'])

    #drop reference electrodes from further analyses
    data_ref.drop_channels(['EXG1', 'EXG2'])

    #set montage
    data_ref.set_montage('standard_1020')
    
    return data_ref

In [4]:
#set filter parameters

def apply_filter(pp_num, root_dir, hcutoff, lcutoff, linefr):
    
    data = set_reference(pp_num, root_dir)
    
    line_freqs = linefr

    #highpass at 0.1 removes slow wave drift
    data_filt = data.filter(l_freq = hcutoff, h_freq= lcutoff)
    data_filt = data_filt.notch_filter(line_freqs, filter_length='auto', phase='zero')

    # plot density to see after filtering
    # raw_filt.compute_psd(fmax=250).plot()
    
    return data_filt

In [ ]:
# # Epoch using target information (same as trial start)
# # First epoch using event_dict. This will create an Epoch around all triggers in event_dict.
# def get_epochs(data):

#     targetinfo = ["target_q1_hor_near","target_q1_hor_mid","target_q1_hor_far","target_q3_hor_near","target_q3_hor_mid","target_q3_hor_far","target_q1_ver_near","target_q1_ver_mid","target_q1_ver_far","target_q3_ver_near","target_q3_ver_mid","target_q3_ver_far","target_q2_hor_near","target_q2_hor_mid","target_q2_hor_far","target_q4_hor_near","target_q4_hor_mid","target_q4_hor_far","target_q2_ver_near","target_q2_ver_mid","target_q2_ver_far","target_q4_ver_near","target_q4_ver_mid","target_q4_ver_far","targetinfo_1","targetinfo_2","targetinfo_3"]
#     event_values = [event_dict[i] for i in targetinfo]
#     epochs = mne.Epochs(data, events, event_id = event_values, on_missing = 'ignore', baseline = None, tmin=-2.5, tmax=10, preload = True)
#     return epochs

In [5]:
# Epoch using feedback onset
# First epoch using event_dict. This will create an Epoch around all triggers in event_dict.
def get_epochs(pp_num, root_dir, hpass, lpass, linefreqs, trigger):
    
    data = apply_filter(pp_num, root_dir, hpass, lpass, linefreqs)
    
    events = mne.find_events(data, initial_event=False)
    
    #marker = ["target_q1_hor_near","target_q1_hor_mid","target_q1_hor_far","target_q3_hor_near","target_q3_hor_mid","target_q3_hor_far","target_q1_ver_near","target_q1_ver_mid","target_q1_ver_far","target_q3_ver_near","target_q3_ver_mid","target_q3_ver_far","target_q2_hor_near","target_q2_hor_mid","target_q2_hor_far","target_q4_hor_near","target_q4_hor_mid","target_q4_hor_far","target_q2_ver_near","target_q2_ver_mid","target_q2_ver_far","target_q4_ver_near","target_q4_ver_mid","target_q4_ver_far","targetinfo_1","targetinfo_2","targetinfo_3"]
    marker = [trigger]
    event_values = [event_dict[i] for i in marker]
    
    epochs = mne.Epochs(data, events, event_id = event_values, on_missing = 'ignore', baseline = None, tmin=-1.5, tmax=1.5, preload = True)
    return epochs

In [6]:
def fit_ICA(data, plot = False):
    # Plot is true or false
    # set up and fit the ICA
    
    ica = mne.preprocessing.ICA(random_state=97, max_iter=1000, method = 'infomax', fit_params=dict(extended=True))
    ica.fit(data)
    
    #ICLabel to get automatic labels for components
    iclabel = label_components(data, ica, method='iclabel')
    
    if plot == True:
        ica.plot_sources(data, show_scrollbars=True, start = 0, stop = 1)
    
    return ica, iclabel

In [7]:
def apply_ICA(threshold, iclabel, data_to_apply, ica, plot = False):

    # Convert ICLabel results to data frame
    IC_df = pd.DataFrame(data=iclabel)
    # Get indices for ICs to remove
    IC_exclude_idx = IC_df[(IC_df.y_pred_proba>threshold)&(IC_df.labels!='other')&(IC_df.labels!='brain')].index.values

    # Mark the selected ICs for exclusion
    ica.exclude = IC_exclude_idx
    
    # Apply the ICA solution to the original data (filtered at 0.1Hz) and save the solution to examine later.
    ica.apply(data_to_apply)
    
    if plot == True:
    # Plot the ICA-excluded Epoched data
        data_to_fit.plot(events = events, n_epochs = 1, event_id = event_dict, event_color = 'red')
    
    return data_to_apply, IC_df

In [8]:
def save_ICA(pp_num, IC_df, ica, ppdir, potential):
    patherp = os.path.join(ppdir, potential)
    is_exist = os.path.exists(patherp)
    if not is_exist:
        os.makedirs(patherp)
    
    # Save the ICA results 
    ica_filename = os.path.join(patherp, 'mra_p%03d_run0-ica.fif' % pp_num)
    ica.save(ica_filename, overwrite = True)

    # Save the iclabel output
    df_filename = os.path.join(patherp, 'mra_p%03d_run0_IClabel.csv' % pp_num)
    IC_df.to_csv(df_filename)

# Uncomment if you want to read ica data back in.
# read_ica = mne.preprocessing.read_ica(ica_filename)

In [9]:
def downsample(data_epoch, freq):

    epochs_ds = data_epoch.resample(sfreq=freq)
    #epochs_ds

    #epochs_ds.plot(events=events, event_color='red')
    #epochs_ds.compute_psd(fmax=50).plot()
    
    return epochs_ds

In [10]:
def output_epochs(pp_num, data, ppdir, potential):
    patherp = os.path.join(ppdir, potential)
    is_exist = os.path.exists(patherp)
    if not is_exist:
        os.makedirs(patherp)
    # Save the cleaned data :)
    out_fname = os.path.join(patherp, 'mra_p%03d_run0-epo.fif' % pp_num)
    data.save(out_fname, overwrite = True)

In [11]:
def prepro_data(pp = range(0,32), linefreqs = (60, 120, 180, 240), hpass = 0.1, hpassICA = 1, lpass = 100, trigger = "reach_onset", potential = "ern", exc_prob = .8, dsfreq = 100):

    for pp_idx in pp:
        raw, pp_directory = load_bdf(pp_idx, root)
        epoch_data = get_epochs(pp_idx, root, hpass, lpass, linefreqs, trigger)
        epoch_ICA = get_epochs(pp_idx, root, hpassICA, lpass, linefreqs, trigger)
    
        ica_dat, labels = fit_ICA(epoch_ICA)
        epochs, IC_export = apply_ICA(exc_prob, labels, epoch_data, ica_dat)
        save_ICA(pp_idx, IC_export, ica_dat, pp_directory, potential)
        epochs_ds = downsample(epochs, dsfreq)
        output_epochs(pp_idx, epochs_ds, pp_directory, potential)
        
        #take note of time when one participant finishes
        now = datetime.now()
        timestamp_format = "%Y-%m-%d %H-%M"
        timestamp_str = now.strftime(timestamp_format)
        
        patherp = os.path.join(pp_directory, potential)
        with open( os.path.join(patherp, 'mra_p%03d_run0-time.txt' % pp_idx), "w") as file:
            file.write(timestamp_str)
    

In [12]:
prepro_data()

Extracting EDF parameters from F:\Documents\Science\MirRevAdaptEEG\data\eeg\p000\mra_p000_run0.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 1561087  =      0.000 ...  3048.998 secs...
Extracting EDF parameters from F:\Documents\Science\MirRevAdaptEEG\data\eeg\p000\mra_p000_run0.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 1561087  =      0.000 ...  3048.998 secs...
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper pas

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s


Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 3381 samples (6.604 sec)



[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    3.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s


Trigger channel has a non-zero initial value of 81664 (consider using initial_event=True to detect this event)
3000 events found
Event IDs: [16138 16139 16140 16141 16142 16143 16144 16148 16149 16150 16151 16152
 16153 16154 16155 16156 16158 16159 16160 16161 16162 16163 16164 16165
 16166 16167 16168 16169 81664]
Not setting metadata


...

Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 3381 samples (6.604 sec)



[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    2.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s


Trigger channel has a non-zero initial value of 81664 (consider using initial_event=True to detect this event)
3000 events found
Event IDs: [16138 16139 16140 16141 16142 16143 16144 16148 16149 16150 16151 16152
 16153 16154 16155 16156 16158 16159 16160 16161 16162 16163 16164 16165
 16166 16167 16168 16169 81664]
Not setting metadata


[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    3.3s finished


420 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 420 events and 1537 original time points ...
0 bad epochs dropped
Fitting ICA to data using 64 channels (please be patient, this may take a while)
Selecting by non-zero PCA components: 64 components
Computing Extended Infomax ICA
Fitting ICA took 49.7s.
Applying ICA to Epochs instance
    Transforming to ICA space (64 components)
    Zeroing out 6 ICA components
    Projecting back using 64 PCA components
Writing ICA solution to F:\Documents\Science\MirRevAdaptEEG\data\eeg\p000\lrp\mra_p000_run0-ica.fif...
